In [21]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET  # Pour parser les fichiers XML
import cv2
from ultralytics import YOLO

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder

import pickle
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

## Data Prep

In [22]:
def extract_color_histogram(image, bins=(8, 8, 8)):
    # Convertir l'image en espace de couleur HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Calculer l'histogramme en 3D pour H, S et V
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
    
    # Normaliser l'histogramme
    cv2.normalize(hist, hist)
    
    return hist.flatten()

In [23]:
def extract_shape_features(image):
    # Convertir en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Seuillage pour obtenir un contour net
    _, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    
    # Trouver les contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Si des contours existent, calculer les moments de Hu
    if contours:
        cnt = max(contours, key=cv2.contourArea)  # Prendre le plus grand contour
        moments = cv2.moments(cnt)
        hu_moments = cv2.HuMoments(moments).flatten()
        
        # Normalisation log
        hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)
        
        return hu_moments
    return None

In [24]:
def extract_hog_features(image):
    # Convertir en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Vérifier que l'image n'est pas vide
    if gray is None or gray.size == 0:
        print("Erreur : Image vide pour HOG")
        return np.array([])  # Retourne un tableau vide si problème

    # Redimensionner l'image à une taille standard (64x128 pour HOG)
    gray = cv2.resize(gray, (64, 128))

    # Définir le descripteur HOG
    hog = cv2.HOGDescriptor()

    # Calculer les descripteurs HOG
    hog_features = hog.compute(gray)

    # Vérifier si le calcul est réussi
    if hog_features is None:
        print("Erreur : Impossible de calculer HOG")
        return np.array([])

    return hog_features.flatten()

In [25]:

# Charger le modèle YOLO
model = YOLO("yolov8n.pt")

# Dossiers
dossier_origin = "train/"
output_folder = "detected_objects"

# Vérifier et créer le dossier de sortie
os.makedirs(output_folder, exist_ok=True)

# Parcourir toutes les images dans le dossier d'origine
for k, nom_fichier in enumerate(os.listdir(dossier_origin)):
    if nom_fichier.endswith(".jpg"): 
        image_path = os.path.join(dossier_origin, nom_fichier)
        image = cv2.imread(image_path)  # Charger l'image

        if image is None:
            print(f"Erreur : Impossible de charger {image_path}")
            continue

        # Faire la détection sur cette image
        results = model(image)

        # Extraire les objets détectés
        for i, (box, cls) in enumerate(zip(results[0].boxes.xyxy, results[0].boxes.cls)):
            x_min, y_min, x_max, y_max = map(int, box)  # Convertir en entiers
            class_name = model.names[int(cls)]  # Obtenir le nom de la classe détectée

            # Rogner l'objet détecté
            cropped_object = image[y_min:y_max, x_min:x_max]

            # Vérifier si le crop est valide
            if cropped_object.shape[0] == 0 or cropped_object.shape[1] == 0:
                print(f"Avertissement : Objet {class_name} ignoré (zone trop petite)")
                continue

            # Construire le chemin du fichier avec le nom de l'image d'origine
            if class_name in {"banana", "apple", "orange"}:
                filename = os.path.join(output_folder, f"{class_name}_{k}_{i}.jpg")

                # Sauvegarder l'image découpée
                cv2.imwrite(filename, cropped_object)
                print(f"Objet détecté enregistré : {filename}")

print("Extraction terminée.")



0: 640x640 1 apple, 2734.6ms
Speed: 18.9ms preprocess, 2734.6ms inference, 26.8ms postprocess per image at shape (1, 3, 640, 640)
Objet détecté enregistré : detected_objects\apple_0_0.jpg

0: 640x640 1 apple, 2758.4ms
Speed: 25.0ms preprocess, 2758.4ms inference, 31.8ms postprocess per image at shape (1, 3, 640, 640)
Objet détecté enregistré : detected_objects\apple_2_0.jpg

0: 448x640 3 apples, 1 carrot, 3456.1ms
Speed: 13.8ms preprocess, 3456.1ms inference, 7.6ms postprocess per image at shape (1, 3, 448, 640)
Objet détecté enregistré : detected_objects\apple_4_0.jpg
Objet détecté enregistré : detected_objects\apple_4_1.jpg
Objet détecté enregistré : detected_objects\apple_4_2.jpg

0: 640x640 2 apples, 1544.7ms
Speed: 25.5ms preprocess, 1544.7ms inference, 25.7ms postprocess per image at shape (1, 3, 640, 640)
Objet détecté enregistré : detected_objects\apple_6_0.jpg
Objet détecté enregistré : detected_objects\apple_6_1.jpg

0: 640x640 3 apples, 1577.6ms
Speed: 13.1ms preprocess, 15

In [26]:

# Dossier contenant les images extraites par YOLO
dossier = "detected_objects"

# Initialisation d'un tableau pour stocker les descripteurs
tab_descripteurs = []

# Parcourir toutes les images du dossier
for nom_fichier in os.listdir(dossier):
    if nom_fichier.endswith(".jpg"): 
        # Charger l'image
        chemin_image = os.path.join(dossier, nom_fichier)
        image = cv2.imread(chemin_image)

        if image is None:
            print(f"Erreur : Impossible de charger {chemin_image}")
            continue  # Passe à l'image suivante

        # Extraire les descripteurs
        color_features = extract_color_histogram(image)
        shape_features = extract_shape_features(image)
        hog_features = extract_hog_features(image)

        # Vérifier que les descripteurs existent
        if shape_features is None or len(shape_features) == 0:
            shape_features = np.array([0])  # Valeur par défaut si aucun contour

        # Concaténer les descripteurs en un seul vecteur
        feature_vector = np.concatenate([color_features, shape_features, hog_features])

        # Ajouter le vecteur au tableau des descripteurs
        tab_descripteurs.append(feature_vector)


# Convertir la liste en un tableau NumPy final
if tab_descripteurs:
    tab_descripteurs = np.vstack(tab_descripteurs)
else:
    tab_descripteurs = np.array([])  # Tableau vide si aucune image n'a été traitée
    print(" Aucun descripteur n'a été extrait.")



test avec un mlp

In [27]:
X = tab_descripteurs  # Ton tableau numpy (features)

labels = [nom.split("_")[0] for nom in os.listdir("detected_objects") if nom.endswith(".jpg")]
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
y = encoded_labels  # Une liste ou array numpy des classes associées

# Normaliser les données (important pour MLP)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Séparer en train/test (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=1)

# Définir le modèle MLP
mlp = MLPClassifier(hidden_layer_sizes=(30, 10), activation='relu', max_iter=500)

# Entraîner le modèle
mlp.fit(X_train, y_train)

# Prédire sur le test set
y_pred = mlp.predict(X_test)

# Évaluer la précision
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
print(f"Accuracy du modèle : {accuracy:.2f}")
print(f"Précision du modèle : {precision:.2f}")



Accuracy du modèle : 0.82
Précision du modèle : 0.82


In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=1)
rf.fit(X, y)  # Apprentissage avant d'intégrer dans le pipeline

feature_selector = SelectFromModel(rf)

# Création du pipeline 
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('FS', feature_selector),
    ('Classifier', mlp)
])

# Entraînement du pipeline
pipeline.fit(X_train, y_train)

# Évaluation du pipeline
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')

print(f"Accuracy du modèle : {accuracy:.2f}")
print(f"Précision du modèle : {precision:.2f}")

# Sauvegarde du pipeline avec pickle
with open("mlp_pipeline.pkl", "wb") as f:
    pickle.dump(pipeline, f)

print("Pipeline sauvegardé avec succès !")

Accuracy du modèle : 0.87
Précision du modèle : 0.87
Pipeline sauvegardé avec succès !


: 